In [1]:
from pathlib import Path

import Metashape

from mynd.backends import metashape as backend
from mynd.utils.log import logger
from mynd.utils.result import Ok, Err, Result

/home/martin/.cache/pypoetry/virtualenvs/mynd-qzh2EXyG-py3.11/lib/python3.11/site-packages/kornia/feature/lightglue.py:44: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
  @torch.cuda.amp.custom_fwd(cast_inputs=torch.float32)


ModuleNotFoundError: No module named 'mynd.backends.metashape.data_types'

### Load document into backend

In [ ]:
DOCUMENT_PATH: Path = Path(
    "/data/kingston_snv_01/acfr_revisits_metashape_projects/r23685bc_working_version.psz"
)

result: Result[str, str] = backend.load_project(DOCUMENT_PATH)
match result:
    case Ok(message):
        logger.info(message)
    case Err(message):
        logger.error(message)

### Backend playground

In [ ]:
result: Result = backend.camera_services.request_stereo_data()

data = result.unwrap()

for key, stereo_groups in data.items():
    logger.info(f"Key: {key}, group count: {len(stereo_groups)}")